# Setttings

In [1]:
import numpy as np
import tensorflow as tf

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Read in data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../Tensorflow-applications/MNIST_data//", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../Tensorflow-applications/MNIST_data//t10k-images-idx3-ubyte.gz
Extracting ../../Tensorflow-applications/MNIST_data//t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(mnist.train.images.shape)
print(mnist.validation.images.shape)
print(mnist.test.images.shape)

(55000, 784)
(5000, 784)
(10000, 784)


In [4]:
mnist.train.labels[:,:].shape

(55000, 10)

# Create graph

### Settings

In [5]:
epsilon = 1e-7
learning_rate = 0.001
batch_size = 50
epochs = 10
rounds = 3

### Input and output tensors

In [6]:
X = tf.placeholder(tf.float32, [None, 784])
X_reshape = tf.reshape(X, [-1, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10])

## Squash function

In [7]:
def squash(input_vector, axis):
    normalised_input = tf.reduce_sum(tf.square(input_vector), axis = axis, keepdims = True)
    scale = tf.divide(normalised_input, tf.add(normalised_input, 1.))
    vector = tf.divide(input_vector, tf.sqrt(tf.add(normalised_input, epsilon)))
    output = tf.multiply(scale, vector)
    
    return(output)

### Layers

#### Convolutional

In [8]:
conv_filter_shape = [9, 9, 1, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W1')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B1')
out_layer = tf.nn.conv2d(X_reshape, weights, [1,1,1,1], padding = 'VALID')
out_layer_bias = tf.add(out_layer, bias)
layer1 = tf.nn.relu(out_layer_bias)

#### PrimaryCaps

In [9]:
conv_filter_shape = [9, 9, 256, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W2')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B2')
out_layer = tf.nn.conv2d(layer1, weights, [1,2,2,1], padding = 'VALID')
layer2 = tf.add(out_layer, bias)

In [10]:
caps1_raw = tf.reshape(layer2, [-1, 6*6*32, 8], name='caps1_raw')
caps1_output = squash(caps1_raw, axis=-1)

In [11]:
caps1_output_expand = tf.expand_dims(caps1_output, axis=-1)
caps1_output_expand2 = tf.expand_dims(caps1_output_expand, axis=2)
caps1_output_expand2_tiled = tf.tile(caps1_output_expand2, [1,1,10,1,1], name = 'caps1_out_tiled')

In [12]:
weight_matrix = tf.Variable(tf.truncated_normal([1, 1152, 10, 16, 8], stddev=0.1), name='W_matrix')
batch = tf.shape(X)[0]
weight_matrix_tiled = tf.tile(weight_matrix, [batch, 1, 1, 1, 1], name = 'W_matrix_tiled')

In [13]:
caps2_predicted = tf.matmul(weight_matrix_tiled, caps1_output_expand2_tiled, name='caps2_predicted')

### Routing by agreement

In [14]:
raw_weights = tf.zeros([batch, 1152, 10, 1, 1], name = 'raw_weights')

In [15]:
for i in range(rounds):
    routing_weights = tf.nn.softmax(raw_weights, axis=2, name = 'routing_weights' + str(i))
    weighted_predictions = tf.multiply(routing_weights, caps2_predicted, name = 'weighted_predictions' + str(i))
    weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, name = 'weighted_sum' + str(i), keepdims = True)
    caps2_output = squash(weighted_sum, axis=-2)
    
    caps2_output_tiled = tf.tile(caps2_output, [1, 1152, 1, 1, 1], name = 'caps2_output_tiled'+ str(i))
    agreement = tf.matmul(caps2_predicted, caps2_output_tiled, transpose_a = True, name = 'agreement'+ str(i))
    raw_weights = tf.add(raw_weights, agreement, name = 'raw_weights' + str(i))

In [16]:
caps2_output

<tf.Tensor 'Mul_3:0' shape=(?, 1, 10, 16, 1) dtype=float32>

In [17]:
squared_norm_caps2 = tf.reduce_sum(tf.square(caps2_output), axis=-2, keepdims = True, name='caps2_norm')
caps2_activation = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation')

In [18]:
caps2_activation

<tf.Tensor 'caps2_activation:0' shape=(?, 1, 10, 1, 1) dtype=float32>

In [19]:
y_estimate = tf.squeeze(caps2_activation, axis=[1,3,4], name = 'y_pred')

In [20]:
y_pred = tf.nn.softmax(y_estimate, axis=1)

### Los function

In [21]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_estimate, labels=y))
optimiser = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

### Make predictions

In [22]:
correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

## Init and saver

In [23]:
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

## Setup training

In [ ]:
restore_checkpoint = True
best_loss_val = np.infty
checkpoint_path = "./my_capsule_network"

#If run on AWS
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth=True


with tf.Session(config=config) as sess:
    #if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
    #    saver.restore(sess, checkpoint_path)
    #else:
    init_op.run()

    total_batch = int(len(mnist.train.images)/batch_size)
    total_batch_validation = int(len(mnist.validation.images)/batch_size)

    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            a, c= sess.run([optimiser, cross_entropy], feed_dict={X: batch_x, y: batch_y})
            avg_cost += c/total_batch
            print('batch:', i)
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        
        acc_vals = []
        for iterations in range(total_batch_validation):
            batch_x, batch_y = mnist.validation.next_batch(batch_size=batch_size)
            val_acc = sess.run(accuracy, feed_dict={X: batch_x, y: batch_y})
            acc_vals.append(val_acc)
        acc_val = np.mean(acc_vals)
        print('val_acc:', acc_val)
        
        # And save the model if it improved:
        #if loss_val < best_loss_val:
        #    save_path = saver.save(sess, checkpoint_path)
        #    best_loss_val = loss_val

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100

batch: 755
batch: 756
batch: 757
batch: 758
batch: 759
batch: 760
batch: 761
batch: 762
batch: 763
batch: 764
batch: 765
batch: 766
batch: 767
batch: 768
batch: 769
batch: 770
batch: 771
batch: 772
batch: 773
batch: 774
batch: 775
batch: 776
batch: 777
batch: 778
batch: 779
batch: 780
batch: 781
batch: 782
batch: 783
batch: 784
batch: 785
batch: 786
batch: 787
batch: 788
batch: 789
batch: 790
batch: 791
batch: 792
batch: 793
batch: 794
batch: 795
batch: 796
batch: 797
batch: 798
batch: 799
batch: 800
batch: 801
batch: 802
batch: 803
batch: 804
batch: 805
batch: 806
batch: 807
batch: 808
batch: 809
batch: 810
batch: 811
batch: 812
batch: 813
batch: 814
batch: 815
batch: 816
batch: 817
batch: 818
batch: 819
batch: 820
batch: 821
batch: 822
batch: 823
batch: 824
batch: 825
batch: 826
batch: 827
batch: 828
batch: 829
batch: 830
batch: 831
batch: 832
batch: 833
batch: 834
batch: 835
batch: 836
batch: 837
batch: 838
batch: 839
batch: 840
batch: 841
batch: 842
batch: 843
batch: 844
batch: 845

batch: 397
batch: 398
batch: 399
batch: 400
batch: 401
batch: 402
batch: 403
batch: 404
batch: 405
batch: 406
batch: 407
batch: 408
batch: 409
batch: 410
batch: 411
batch: 412
batch: 413
batch: 414
batch: 415
batch: 416
batch: 417
batch: 418
batch: 419
batch: 420
batch: 421
batch: 422
batch: 423
batch: 424
batch: 425
batch: 426
batch: 427
batch: 428
batch: 429
batch: 430
batch: 431
batch: 432
batch: 433
batch: 434
batch: 435
batch: 436
batch: 437
batch: 438
batch: 439
batch: 440
batch: 441
batch: 442
batch: 443
batch: 444
batch: 445
batch: 446
batch: 447
batch: 448
batch: 449
batch: 450
batch: 451
batch: 452
batch: 453
batch: 454
batch: 455
batch: 456
batch: 457
batch: 458
batch: 459
batch: 460
batch: 461
batch: 462
batch: 463
batch: 464
batch: 465
batch: 466
batch: 467
batch: 468
batch: 469
batch: 470
batch: 471
batch: 472
batch: 473
batch: 474
batch: 475
batch: 476
batch: 477
batch: 478
batch: 479
batch: 480
batch: 481
batch: 482
batch: 483
batch: 484
batch: 485
batch: 486
batch: 487

batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100
batch: 101
batch: 102
batch: 103
batch: 104
batch: 105
batch: 106
batch: 107
batch: 108
batch: 109
batch: 110
batch: 111
batch: 112
batch: 113
batch: 114
batch: 115
batch: 116
batch: 117
batch: 118
batch: 119
batch: 120
batch: 121
batch: 122
batch: 123
batch: 124
batch: 125
batch: 126
batch: 127
batch: 128
batch: 129


batch: 784
batch: 785
batch: 786
batch: 787
batch: 788
batch: 789
batch: 790
batch: 791
batch: 792
batch: 793
batch: 794
batch: 795
batch: 796
batch: 797
batch: 798
batch: 799
batch: 800
batch: 801
batch: 802
batch: 803
batch: 804
batch: 805
batch: 806
batch: 807
batch: 808
batch: 809
batch: 810
batch: 811
batch: 812
batch: 813
batch: 814
batch: 815
batch: 816
batch: 817
batch: 818
batch: 819
batch: 820
batch: 821
batch: 822
batch: 823
batch: 824
batch: 825
batch: 826
batch: 827
batch: 828
batch: 829
batch: 830
batch: 831
batch: 832
batch: 833
batch: 834
batch: 835
batch: 836
batch: 837
batch: 838
batch: 839
batch: 840
batch: 841
batch: 842
batch: 843
batch: 844
batch: 845
batch: 846
batch: 847
batch: 848
batch: 849
batch: 850
batch: 851
batch: 852
batch: 853
batch: 854
batch: 855
batch: 856
batch: 857
batch: 858
batch: 859
batch: 860
batch: 861
batch: 862
batch: 863
batch: 864
batch: 865
batch: 866
batch: 867
batch: 868
batch: 869
batch: 870
batch: 871
batch: 872
batch: 873
batch: 874

batch: 426
batch: 427
batch: 428
batch: 429
batch: 430
batch: 431
batch: 432
batch: 433
batch: 434
batch: 435
batch: 436
batch: 437
batch: 438
batch: 439
batch: 440
batch: 441
batch: 442
batch: 443
batch: 444
batch: 445
batch: 446
batch: 447
batch: 448
batch: 449
batch: 450
batch: 451
batch: 452
batch: 453
batch: 454
batch: 455
batch: 456
batch: 457
batch: 458
batch: 459
batch: 460
batch: 461
batch: 462
batch: 463
batch: 464
batch: 465
batch: 466
batch: 467
batch: 468
batch: 469
batch: 470
batch: 471
batch: 472
batch: 473
batch: 474
batch: 475
batch: 476
batch: 477
batch: 478
batch: 479
batch: 480
batch: 481
batch: 482
batch: 483
batch: 484
batch: 485
batch: 486
batch: 487
batch: 488
batch: 489
batch: 490
batch: 491
batch: 492
batch: 493
batch: 494
batch: 495
batch: 496
batch: 497
batch: 498
batch: 499
batch: 500
batch: 501
batch: 502
batch: 503
batch: 504
batch: 505
batch: 506
batch: 507
batch: 508
batch: 509
batch: 510
batch: 511
batch: 512
batch: 513
batch: 514
batch: 515
batch: 516

batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100
batch: 101
batch: 102
batch: 103
batch: 104
batch: 105
batch: 106
batch: 107
batch: 108
batch: 109
batch: 110
batch: 111
batch: 112
batch: 113
batch: 114
batch: 115
batch: 116
batch: 117
batch: 118
batch: 119
batch: 120
batch: 121
batch: 122
batch: 123
batch: 124
batch: 125
batch: 126
batch: 127
batch: 128
batch: 129
batch: 130
batch: 131
batch: 132
batch: 133
batch: 134
batch: 135
batch: 136
batch: 137
batch: 138
batch: 139
batch: 140
batch: 141
batch: 142
batch: 143
batch: 144
batch: 145
batch: 146
batch: 147
batch: 148
batch: 149
batch: 150
batch: 151
batch: 152
batch: 153
batch: 154
batch: 155
batch: 156
batch: 157
batch: 158
batch: 159
